In [3]:
!pip install sentence_transformers

In [4]:
import json
import re
from spacy import load as spacyload
from sentence_transformers import SentenceTransformer


In [5]:
def tokenize(doc, lemmatized=False, remove_stopword=False, 
                   remove_punct = True, pos_tag = False, remove_num=False):
    nlp=spacyload("en_core_web_sm")
    doc=nlp(doc.lower().strip())

    tokens=[]

    for token in doc:
        if (remove_stopword and token.is_stop) or (remove_punct and token.is_punct) or (token.text=='') or (token.text.isspace()) or (remove_num and token.text.isnumeric()):
            continue

        token_ls=token.text
        if lemmatized:
            token_ls=token.lemma_
        if pos_tag:
            token_ls=(token_ls, token.pos_)
        tokens.append(token_ls)
    return tokens


In [8]:
patent_file=json.load(open("./dataset/US-11604547-B2/US-11604547-B2.json"))

In [9]:
# re.split(r"([0-9]+\.)", patent_file['claims2']) # Use this regex if you want claim number included in the list. 

claims = [ claim.replace("\n", " ")for claim in re.split(r"[0-9]+\.", patent_file['claims2']) if claim!='']
claims = [ ' '.join(tokenize(doc=claim, lemmatized=True, remove_punct=True, remove_stopword=True, remove_num=True)) for claim in claims ]


In [10]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(claims, show_progress_bar=True, convert_to_numpy=True)
print(embeddings.shape)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(20, 768)
